In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Any, Optional, TypedDict
from dataclasses import dataclass
import re

In [ ]:
# Deep Learning & NLP
from transformers import (
    TrainingArguments,
    Trainer,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from datasets import Dataset

In [ ]:
# LangChain & LangGraph
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langgraph.graph import StateGraph, END

# Utilities
from sentence_transformers import CrossEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(" All libraries imported successfully!")
print(f" PyTorch version: {torch.__version__}")
print(f" CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"💻 GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
@dataclass
class ProjectConfig:
    BASE_DIR: Path = Path("../../JupyterProject")
    DATA_DIR: Path = BASE_DIR / "data"
    MODEL_DIR: Path = BASE_DIR / "models"
    RESULTS_DIR: Path = BASE_DIR / "results"
    VECTOR_DB_DIR: Path = BASE_DIR / "vector_db"
    BASE_MODEL: str = "mistralai/Mistral-7B-Instruct-v0.2"
    EMBEDDING_MODEL: str = "sentence-transformers/all-MiniLM-L6-v2"
    RERANKER_MODEL: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    CHUNK_SIZE: int = 500
    CHUNK_OVERLAP: int = 50
    TOP_K_RETRIEVAL: int = 5
    RERANK_TOP_K: int = 3
    # Fine-tuning settings
    LORA_R: int = 16
    LORA_ALPHA: int = 32
    LORA_DROPOUT: float = 0.05
    LEARNING_RATE: float = 2e-4
    BATCH_SIZE: int = 4
    GRADIENT_ACCUMULATION_STEPS: int = 4
    NUM_EPOCHS: int = 3
    MAX_SEQ_LENGTH: int = 2048

    # Agent settings
    MAX_ITERATIONS: int = 5
    TEMPERATURE: float = 0.7

    def __post_init__(self):
        """Create necessary directories"""
        for directory in [self.DATA_DIR, self.MODEL_DIR,
                self.VECTOR_DB_DIR, self.RESULTS_DIR]:
            directory.mkdir(parents=True, exist_ok=True)

config = ProjectConfig()
print(f"📁 Project initialized at: {config.BASE_DIR}")

In [ ]:
class ResumeParser:
    """Parse and extract structured information from resumes"""

    def __init__(self):
        self.skills_keywords = [
            'python', 'java', 'javascript', 'react', 'sql', 'machine learning',
            'deep learning', 'nlp', 'computer vision', 'aws', 'docker', 'kubernetes'
        ]

    def parse_pdf(self, pdf_path: str) -> Dict[str, Any]:
        """Extract text and metadata from PDF resume"""
        from pypdf import PdfReader

        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

        return {
            'text': text,
            'num_pages': len(reader.pages),
            'metadata': self._extract_metadata(text)
        }

    def _extract_metadata(self, text: str) -> Dict[str, Any]:
        """Extract structured data from resume text"""
        metadata = {
            'email': self._extract_email(text),
            'phone': self._extract_phone(text),
            'skills': self._extract_skills(text),
            'education': self._extract_education(text),
            'experience_years': self._estimate_experience(text)
        }
        return metadata

    def _extract_email(self, text: str) -> Optional[str]:
        """Extract email address"""
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        matches = re.findall(email_pattern, text)
        return matches[0] if matches else None

    def _extract_phone(self, text: str) -> Optional[str]:
        """Extract phone number"""
        phone_pattern = r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'
        matches = re.findall(phone_pattern, text)
        return matches[0] if matches else None

    def _extract_skills(self, text: str) -> List[str]:
        """Extract technical skills"""
        text_lower = text.lower()
        found_skills = [skill for skill in self.skills_keywords
                       if skill in text_lower]
        return found_skills

    def _extract_education(self, text: str) -> List[str]:
        """Extract education degrees"""
        degrees = ['phd', 'ph.d', 'master', 'bachelor', 'mba', 'ms', 'bs', 'ba']
        text_lower = text.lower()
        found_degrees = [deg for deg in degrees if deg in text_lower]
        return list(set(found_degrees))

    def _estimate_experience(self, text: str) -> int:
        """Estimate years of experience"""
        # Look for patterns like "5 years", "2+ years", etc.
        pattern = r'(\d+)\+?\s*years?'
        matches = re.findall(pattern, text.lower())
        if matches:
            return max([int(m) for m in matches])
        return 0

In [ ]:
def generate_synthetic_resumes(n_samples: int = 100) -> pd.DataFrame:
    """Generate synthetic resume data for training"""

    job_titles = ['Software Engineer', 'Data Scientist', 'Product Manager',
                  'DevOps Engineer', 'ML Engineer', 'Frontend Developer']

    skills_pool = {
        'Software Engineer': ['Python', 'Java', 'JavaScript', 'React', 'Node.js', 'SQL', 'Git'],
        'Data Scientist': ['Python', 'R', 'SQL', 'Machine Learning', 'Pandas', 'Scikit-learn', 'TensorFlow'],
        'Product Manager': ['Agile', 'Jira', 'User Research', 'Roadmapping', 'SQL', 'Analytics'],
        'DevOps Engineer': ['Docker', 'Kubernetes', 'AWS', 'CI/CD', 'Terraform', 'Linux', 'Python'],
        'ML Engineer': ['Python', 'TensorFlow', 'PyTorch', 'MLOps', 'Docker', 'AWS', 'Kubernetes'],
        'Frontend Developer': ['JavaScript', 'React', 'Vue.js', 'HTML', 'CSS', 'TypeScript', 'Webpack']
    }

    data = []
    for i in range(n_samples):
        job_title = np.random.choice(job_titles)
        experience_years = np.random.randint(0, 15)
        education = np.random.choice(['Bachelor', 'Master', 'PhD'])
        skills = np.random.choice(skills_pool[job_title],
                                 size=np.random.randint(3, 7),
                                 replace=False).tolist()

        # Generate resume text
        resume_text = f"""
        Professional Summary:
        {job_title} with {experience_years} years of experience in software development.

        Education:
        {education}'s Degree in Computer Science

        Skills:
        {', '.join(skills)}

        Experience:
        - Led development teams and delivered multiple projects
        - Implemented best practices and coding standards
        - Collaborated with cross-functional teams
        """

        data.append({
            'resume_id': f'RES_{i:04d}',
            'job_title': job_title,
            'experience_years': experience_years,
            'education': education,
            'skills': skills,
            'resume_text': resume_text.strip()
        })

    return pd.DataFrame(data)

def generate_job_descriptions(n_samples: int = 50) -> pd.DataFrame:
    """Generate synthetic job descriptions"""

    job_titles = ['Software Engineer', 'Data Scientist', 'Product Manager',
                  'DevOps Engineer', 'ML Engineer', 'Frontend Developer']

    data = []
    for i in range(n_samples):
        job_title = np.random.choice(job_titles)
        required_exp = np.random.randint(2, 10)

        jd_text = f"""
        Position: {job_title}

        Requirements:
        - {required_exp}+ years of experience in relevant field
        - Bachelor's degree in Computer Science or related field
        - Strong problem-solving and communication skills

        Responsibilities:
        - Design and implement scalable solutions
        - Collaborate with team members
        - Mentor junior developers
        """

        data.append({
            'job_id': f'JOB_{i:04d}',
            'job_title': job_title,
            'required_experience': required_exp,
            'job_description': jd_text.strip()
        })

    return pd.DataFrame(data)

# Generate datasets
print(" Generating synthetic datasets...")
resumes_df = generate_synthetic_resumes(100)
jobs_df = generate_job_descriptions(50)

print(f" Generated {len(resumes_df)} resumes and {len(jobs_df)} job descriptions")
print("\n Sample Resume:")
print(resumes_df.iloc[0]['resume_text'][:300] + "...")
print(f"\n Sample Job Description:")
print(jobs_df.iloc[0]['job_description'][:300] + "...")

# Save datasets
resumes_df.to_csv(config.DATA_DIR / 'resumes.csv', index=False)
jobs_df.to_csv(config.DATA_DIR / 'job_descriptions.csv', index=False)

In [ ]:
class DocumentProcessor:
    """Process and chunk documents for RAG"""

    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len
        )

    def process_resumes(self, resumes_df: pd.DataFrame) -> List[Document]:
        """Convert resumes to Document objects with metadata"""
        documents = []

        for idx, row in resumes_df.iterrows():
            doc = Document(
                page_content=row['resume_text'],
                metadata={
                    'type': 'resume',
                    'resume_id': row['resume_id'],
                    'job_title': row['job_title'],
                    'experience_years': row['experience_years'],
                    'education': row['education'],
                    'skills': ','.join(row['skills'])
                }
            )
            documents.append(doc)

        return documents

    def process_job_descriptions(self, jobs_df: pd.DataFrame) -> List[Document]:
        """Convert job descriptions to Document objects"""
        documents = []

        for idx, row in jobs_df.iterrows():
            doc = Document(
                page_content=row['job_description'],
                metadata={
                    'type': 'job_description',
                    'job_id': row['job_id'],
                    'job_title': row['job_title'],
                    'required_experience': row['required_experience']
                }
            )
            documents.append(doc)

        return documents

    def chunk_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks"""
        return self.text_splitter.split_documents(documents)

# Process documents
print(" Processing documents for RAG...")
processor = DocumentProcessor(
    chunk_size=config.CHUNK_SIZE,
    chunk_overlap=config.CHUNK_OVERLAP
)

resume_docs = processor.process_resumes(resumes_df)
job_docs = processor.process_job_descriptions(jobs_df)
all_docs = resume_docs + job_docs

# Chunk documents
chunked_docs = processor.chunk_documents(all_docs)
print(f"Created {len(chunked_docs)} document chunks from {len(all_docs)} documents")
print(f"Average chunk size: {np.mean([len(doc.page_content) for doc in chunked_docs]):.0f} characters")


In [ ]:
class HybridRetriever:
    """Hybrid retrieval system combining dense and sparse retrieval"""

    def __init__(self, documents: List[Document], config: ProjectConfig):
        self.config = config
        self.documents = documents

        # Dense retrieval (semantic)
        print("Initializing dense retriever (embeddings)...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name= config.EMBEDDING_MODEL,
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
        )

        self.vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=str(config.VECTOR_DB_DIR)
        )

        # Sparse retrieval (BM25)
        print("Initializing sparse retriever (BM25)...")
        from rank_bm25 import BM25Okapi
        tokenized_docs = [doc.page_content.lower().split() for doc in documents]
        self.bm25 = BM25Okapi(tokenized_docs)

        # Re-ranker
        print("Loading re-ranker model...")
        self.reranker = CrossEncoder(config.RERANKER_MODEL)

        print("Hybrid retriever initialized!")

    def retrieve(self, query: str, k: int = 5) -> List[Document]:
        """Hybrid retrieval with re-ranking"""

        # Dense retrieval
        dense_results = self.vectorstore.similarity_search(query, k=k*2)

        # Sparse retrieval
        tokenized_query = query.lower().split()
        bm25_scores = self.bm25.get_scores(tokenized_query)
        top_bm25_indices = np.argsort(bm25_scores)[-k*2:][::-1]
        sparse_results = [self.documents[i] for i in top_bm25_indices]

        # Combine and deduplicate
        combined_results = list({doc.page_content: doc
                                for doc in dense_results + sparse_results}.values())

        # Re-rank
        if len(combined_results) > k:
            pairs = [[query, doc.page_content] for doc in combined_results]
            scores = self.reranker.predict(pairs)
            top_indices = np.argsort(scores)[-k:][::-1]
            combined_results = [combined_results[i] for i in top_indices]

        return combined_results[:k]

# Initialize retriever
print("\nBuilding Hybrid RAG System...")
retriever = HybridRetriever(chunked_docs, config)

# Test retrieval
test_query = "Senior software engineer with Python and machine learning experience"
retrieved_docs = retriever.retrieve(test_query, k=3)

print(f"\nTest Query: '{test_query}'")
print(f"Retrieved {len(retrieved_docs)} relevant documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{i}. Type: {doc.metadata.get('type', 'unknown')}")
    print(f"   Content: {doc.page_content[:150]}...")

In [ ]:
## 4.1 Prepare Training Data

def create_training_dataset(resumes_df: pd.DataFrame, jobs_df: pd.DataFrame) -> Dataset:
    """Create instruction-tuning dataset for hiring tasks"""

    training_samples = []

    # Task 1: Resume Analysis
    for _, resume in resumes_df.iterrows():
        job = jobs_df.sample(1).iloc[0]

        instruction = "Analyze the following resume against the job requirements and provide a match score with explanation."
        input_text = f"Resume:\n{resume['resume_text']}\n\nJob Description:\n{job['job_description']}"

        # Generate synthetic output
        skill_match = len(set(resume['skills'])) / 7.0  # Normalize
        exp_match = min(resume['experience_years'] / job['required_experience'], 1.0)
        overall_score = int((skill_match * 0.6 + exp_match * 0.4) * 100)

        output = f"""Match Score: {overall_score}/100

Strengths:
- {resume['experience_years']} years of relevant experience
- Skills: {', '.join(resume['skills'][:3])}
- Education: {resume['education']}'s degree

Recommendation: {"STRONG MATCH" if overall_score > 70 else "MODERATE MATCH" if overall_score > 50 else "WEAK MATCH"}"""

        training_samples.append({
            'instruction': instruction,
            'input': input_text,
            'output': output
        })

    # Task 2: Interview Question Generation
    for _, job in jobs_df[:20].iterrows():
        instruction = "Generate 5 technical interview questions for this role."
        input_text = f"Job Title: {job['job_title']}\n\n{job['job_description']}"

        output = f"""Interview Questions for {job['job_title']}:

1. Technical: Describe your experience with the core technologies required for this role.
2. Problem-Solving: Walk me through how you would approach [specific technical challenge].
3. Experience: Tell me about a complex project you've delivered. What was your role?
4. Collaboration: How do you handle disagreements with team members on technical decisions?
5. Growth: What areas are you currently working to improve in your skillset?"""

        training_samples.append({
            'instruction': instruction,
            'input': input_text,
            'output': output
        })

    # Task 3: Skills Gap Analysis
    for _, resume in resumes_df[:20].iterrows():
        job = jobs_df.sample(1).iloc[0]

        instruction = "Identify skills gaps between the candidate and role requirements."
        input_text = f"Candidate Skills: {', '.join(resume['skills'])}\n\nRequired for: {job['job_title']}"

        output = f"""Skills Gap Analysis:

Current Skills: {', '.join(resume['skills'][:4])}
Missing Critical Skills: [Based on role requirements]
Nice-to-Have Skills: [Additional beneficial skills]

Recommendation: Focus on developing [specific skills] through online courses or projects."""

        training_samples.append({
            'instruction': instruction,
            'input': input_text,
            'output': output
        })

    return Dataset.from_list(training_samples)

# Create training dataset
print("Creating instruction-tuning dataset...")
train_dataset = create_training_dataset(resumes_df, jobs_df)
print(f"Created {len(train_dataset)} training examples")
print(f"\n Sample training example:")
print(f"Instruction: {train_dataset[0]['instruction']}")
print(f"Input: {train_dataset[0]['input'][:150]}...")
print(f"Output: {train_dataset[0]['output'][:150]}...")

In [ ]:
def format_prompt(sample: Dict[str, str]) -> str:
    """Format sample into instruction-following prompt"""
    return f"""<s>[INST] {sample['instruction']}

{sample['input']} [/INST] {sample['output']}</s>"""

def tokenize_function(examples, tokenizer, max_length: int = 2048):
    """Tokenize examples for training"""
    prompts = [format_prompt({"instruction": inst, "input": inp, "output": out})
               for inst, inp, out in zip(examples["instruction"],
                                         examples["input"],
                                         examples["output"])]

    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=max_length,
        padding="max_length",
        return_tensors="pt"
    )

    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

In [ ]:
## 4.4 Configure LoRA for fine tuning

def setup_lora(model, config: ProjectConfig):
    """Configure and apply LoRA to the model"""

    print("Configuring LoRA...")

    # LoRA configuration
    lora_config = LoraConfig(
        r=config.LORA_R,
        lora_alpha=config.LORA_ALPHA,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                       "gate_proj", "up_proj", "down_proj"],
        lora_dropout=config.LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Prepare model for training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

    # Print trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    print(f" LoRA configured successfully!")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")

    return model

In [ ]:
def train_model(model, tokenizer, train_dataset, config: ProjectConfig):
    """Fine-tune model with LoRA"""

    print("Starting model training...")

    # Tokenize dataset
    tokenized_dataset = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer, config.MAX_SEQ_LENGTH),
        batched=True,
        remove_columns=train_dataset.column_names
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=str(config.MODEL_DIR / "checkpoints"),
        num_train_epochs=config.NUM_EPOCHS,
        per_device_train_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        learning_rate=config.LEARNING_RATE,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
        optim="paged_adamw_8bit",
        warmup_steps=50,
        lr_scheduler_type="cosine",
        report_to="none"
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer
    )

    # Train
    trainer.train()

    print("Training completed!")

    # Save model
    model.save_pretrained(config.MODEL_DIR / "final_model")
    tokenizer.save_pretrained(config.MODEL_DIR / "final_model")

    return model, tokenizer

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_base_model(config: ProjectConfig):

    print(f"Loading CPU model: {config.BASE_MODEL}")
    print("Using 16-bit (no quantization - CPU safe)")

    model = AutoModelForCausalLM.from_pretrained(
        config.BASE_MODEL,
        dtype=torch.float16,
        device_map={"": "cpu"},
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        config.BASE_MODEL,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    print(" CPU model loaded successfully!")
    print(f" Model size: {model.get_memory_footprint() / 1e9:.2f} GB")
    print(f"🖥  Running on: CPU (16-bit, 8GB RAM)")

    return model, tokenizer

In [ ]:
# NOTE: Due to computational constraints in this notebook, we'll simulate training My pc is dying HAHA!
'''

# Load and prepare model
base_model, tokenizer = load_base_model(config)
lora_model = setup_lora(base_model, config)

# Fine-tune
fine_tuned_model, tokenizer = train_model(lora_model, tokenizer, train_dataset, config)
'''
'''
this is where my pc is shutting down and won't complete this is for testing
import os
from transformers import AutoModelForCausalLM
import time

def safe_download_model(model_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt + 1}/{max_retries}")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                timeout=600,
                resume_download=True,
                cache_dir="./hf_cache",
                low_cpu_mem_usage=True
            )
            print("✅ Download successful!")
            return model
        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(10)

    raise Exception("All download attempts failed")
'''
# and this is for real life training maybe it will need some changes
#model = safe_download_model(config.BASE_MODEL)
# model = AutoModelForCausalLM.from_pretrained(config.BASE_MODEL)
# tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL)

In [ ]:
class AgentState(TypedDict):
    """State object passed between agents"""
    messages: List[str]
    resume_text: Optional[str]
    job_description: Optional[str]
    retrieved_contexts: List[Dict]
    resume_analysis: Optional[Dict]
    interview_questions: Optional[List[str]]
    skills_gaps: Optional[Dict]
    current_agent: str
    iteration: int

In [ ]:
class ResumeAnalyzerAgent:
    """Analyzes resumes against job requirements"""

    def __init__(self, retriever: HybridRetriever, model=None):
        self.retriever = retriever
        self.model = model

    def analyze(self, state: AgentState) -> AgentState:
        """Analyze resume against job description"""

        print("Resume Analyzer Agent activated...")

        resume_text = state.get("resume_text", "")
        job_desc = state.get("job_description", "")

        if not resume_text or not job_desc:
            state["resume_analysis"] = {"error": "Missing resume or job description"}
            return state

        # Retrieve relevant context
        query = f"Resume requirements for: {job_desc[:200]}"
        contexts = self.retriever.retrieve(query, k=config.TOP_K_RETRIEVAL)
        state["retrieved_contexts"] = [
            {"content": doc.page_content, "metadata": doc.metadata}
            for doc in contexts
        ]

        # Parse resume
        parser = ResumeParser()
        resume_metadata = parser._extract_metadata(resume_text)# update this needs to be removed

        # Simple rule-based scoring (in production, use fine-tuned model)
        skills_found = resume_metadata.get('skills', [])
        experience = resume_metadata.get('experience_years', 0)

        score = min(100, len(skills_found) * 15 + experience * 5)

        analysis = {
            "overall_score": score,
            "skills_found": skills_found,
            "experience_years": experience,
            "education": resume_metadata.get('education', []),
            "recommendation": "Strong Match" if score > 70 else "Moderate Match" if score > 50 else "Weak Match",
            "strengths": [
                f"{experience} years of experience",
                f"Skills: {', '.join(skills_found[:3])}" if skills_found else "N/A"
            ],
            "areas_for_review": [
                "Verify specific project experience",
                "Check cultural fit during interview"
            ]
        }

        state["resume_analysis"] = analysis
        state["messages"].append(f"Resume Analysis Complete: {score}/100")

        print(f"Analysis complete. Score: {score}/100")

        return state

In [ ]:
import random
from typing import Dict, List, Any
from dataclasses import dataclass


@dataclass
class QuestionTemplate:
    """Template for generating diverse questions"""
    templates: List[str]
    category: str
    difficulty: str
class InterviewQuestionBank:


    # Technical Question Templates - Multiple variations
    TECHNICAL_TEMPLATES = {
        'experience': [
            "Can you walk me through a challenging project where you used {skill}?",
            "Tell me about your most impactful work with {skill}. What was the outcome?",
            "Describe a situation where {skill} was critical to solving a business problem.",
            "What's the most complex implementation you've done using {skill}?",
            "How have you applied {skill} to improve system performance or user experience?",
        ],
        'problem_solving': [
            "How would you architect a solution using {skill} for [specific scenario]?",
            "If you had to optimize a {skill}-based system serving millions of users, what would you focus on?",
            "Walk me through how you'd debug a performance issue in a {skill} application.",
            "What trade-offs would you consider when choosing {skill} vs alternative technologies?",
            "Describe your approach to testing and validating {skill} implementations.",
        ],
        'depth': [
            "What are the key limitations or challenges of {skill}, and how do you work around them?",
            "How does {skill} work under the hood? Explain the core concepts.",
            "What's a recent update or feature in {skill} that you find interesting? Why?",
            "Compare {skill} with similar technologies. When would you choose one over the other?",
            "What best practices do you follow when working with {skill}?",
        ],
        'practical': [
            "Show me (or describe) how you would implement [specific feature] using {skill}.",
            "What tools and workflows do you use alongside {skill} in your development process?",
            "How do you stay current with {skill} developments and best practices?",
            "Describe a mistake you made with {skill} and what you learned from it.",
            "What metrics do you track when working with {skill} in production?",
        ]
    }

    # Behavioral Question Templates by Theme
    BEHAVIORAL_TEMPLATES = {
        'collaboration': [
            "Tell me about a time when you had to collaborate with a difficult team member. How did you handle it?",
            "Describe a situation where you had to explain a complex technical concept to non-technical stakeholders.",
            "Give an example of when you received critical feedback. How did you respond?",
            "Tell me about a time when your team disagreed on a technical approach. What was your role?",
            "Describe how you've mentored or helped junior developers grow.",
        ],
        'problem_solving': [
            "Walk me through the most challenging technical problem you've solved. What was your approach?",
            "Tell me about a time when you had to make a critical decision with incomplete information.",
            "Describe a situation where you had to fix a critical production bug under pressure.",
            "Give an example of when you identified and solved a problem proactively before it became critical.",
            "Tell me about a time when your initial solution didn't work. How did you adapt?",
        ],
        'leadership': [
            "Describe a time when you had to lead a project or initiative. What was the outcome?",
            "Tell me about a situation where you had to influence others without having formal authority.",
            "Give an example of when you took ownership of a failing project and turned it around.",
            "Describe how you've handled competing priorities from multiple stakeholders.",
            "Tell me about a time when you had to make an unpopular decision. How did you handle it?",
        ],
        'adaptability': [
            "Tell me about a time when project requirements changed significantly mid-development.",
            "Describe a situation where you had to quickly learn a new technology or domain.",
            "Give an example of when you had to work with legacy code or a system you didn't build.",
            "Tell me about a time when you failed. What did you learn?",
            "Describe how you've adapted to a major organizational or team change.",
        ],
        'time_management': [
            "Tell me about a time when you had to manage multiple high-priority tasks. How did you prioritize?",
            "Describe a situation where you had to deliver a project with a very tight deadline.",
            "Give an example of when you had to say 'no' to a request to protect project quality.",
            "Tell me about how you balance technical debt with feature development.",
            "Describe your approach to estimating and managing complex projects.",
        ]
    }

    # Situational Questions by Scenario Type
    SITUATIONAL_TEMPLATES = {
        'system_design': [
            "How would you design a system to handle {specific_requirement}?",
            "If you had to scale {system_type} to 10x current load, what would be your approach?",
            "Walk me through how you'd architect a solution for {use_case}.",
            "What factors would you consider when designing {specific_feature}?",
            "How would you ensure reliability and fault tolerance in {system_context}?",
        ],
        'incident_response': [
            "If you discovered a security vulnerability in production, what would be your immediate steps?",
            "How would you handle a situation where multiple critical systems are failing simultaneously?",
            "What would you do if a deploy caused a 50% increase in error rates?",
            "If a key team member leaves during a critical project phase, how would you handle it?",
            "How would you respond to a customer reporting data corruption in production?",
        ],
        'trade_offs': [
            "How do you balance code quality with delivery speed when under tight deadlines?",
            "When would you choose to refactor existing code vs building new functionality?",
            "How do you decide between building, buying, or using open-source solutions?",
            "What's your approach to managing technical debt while delivering features?",
            "How do you balance perfection with pragmatism in your work?",
        ],
        'team_dynamics': [
            "How would you handle a situation where a team member consistently misses deadlines?",
            "What would you do if you strongly disagreed with your manager's technical decision?",
            "How would you approach code reviews to be constructive without demotivating teammates?",
            "If two team members have conflicting ideas about architecture, how would you facilitate resolution?",
            "How would you onboard a new developer to your codebase and team practices?",
        ]
    }

    # Role-specific specialized questions
    ROLE_SPECIFIC = {
        'Software Engineer': [
            "How do you approach writing testable, maintainable code?",
            "Describe your ideal code review process.",
            "What's your experience with microservices vs monolithic architectures?",
            "How do you ensure backward compatibility when updating APIs?",
        ],
        'Data Scientist': [
            "How do you validate that a machine learning model is ready for production?",
            "Describe your approach to feature engineering for [specific problem].",
            "How do you handle imbalanced datasets?",
            "What's your process for explaining model predictions to stakeholders?",
        ],
        'DevOps Engineer': [
            "How would you implement zero-downtime deployments?",
            "Describe your approach to monitoring and alerting strategies.",
            "How do you handle secrets management in CI/CD pipelines?",
            "What's your strategy for infrastructure as code testing?",
        ],
        'ML Engineer': [
            "How do you monitor and detect model drift in production?",
            "Describe your approach to A/B testing ML models.",
            "How do you optimize model inference latency?",
            "What's your strategy for versioning and reproducing ML experiments?",
        ]
    }


"""
Enhanced Interview Assistant Agent with Dynamic Question Generation
===================================================================
Generates diverse, context-aware interview questions that adapt to:
- Candidate's skill level
- Specific technologies
- Experience gaps
- Role requirements
"""

import random
from typing import Dict, List, Any
from dataclasses import dataclass


@dataclass
class QuestionTemplate:
    """Template for generating diverse questions"""
    templates: List[str]
    category: str
    difficulty: str


class InterviewQuestionBank:
    """Rich bank of diverse question templates"""

    # Technical Question Templates - Multiple variations
    TECHNICAL_TEMPLATES = {
        'experience': [
            "Can you walk me through a challenging project where you used {skill}?",
            "Tell me about your most impactful work with {skill}. What was the outcome?",
            "Describe a situation where {skill} was critical to solving a business problem.",
            "What's the most complex implementation you've done using {skill}?",
            "How have you applied {skill} to improve system performance or user experience?",
        ],
        'problem_solving': [
            "How would you architect a solution using {skill} for [specific scenario]?",
            "If you had to optimize a {skill}-based system serving millions of users, what would you focus on?",
            "Walk me through how you'd debug a performance issue in a {skill} application.",
            "What trade-offs would you consider when choosing {skill} vs alternative technologies?",
            "Describe your approach to testing and validating {skill} implementations.",
        ],
        'depth': [
            "What are the key limitations or challenges of {skill}, and how do you work around them?",
            "How does {skill} work under the hood? Explain the core concepts.",
            "What's a recent update or feature in {skill} that you find interesting? Why?",
            "Compare {skill} with similar technologies. When would you choose one over the other?",
            "What best practices do you follow when working with {skill}?",
        ],
        'practical': [
            "Show me (or describe) how you would implement [specific feature] using {skill}.",
            "What tools and workflows do you use alongside {skill} in your development process?",
            "How do you stay current with {skill} developments and best practices?",
            "Describe a mistake you made with {skill} and what you learned from it.",
            "What metrics do you track when working with {skill} in production?",
        ]
    }

    # Behavioral Question Templates by Theme
    BEHAVIORAL_TEMPLATES = {
        'collaboration': [
            "Tell me about a time when you had to collaborate with a difficult team member. How did you handle it?",
            "Describe a situation where you had to explain a complex technical concept to non-technical stakeholders.",
            "Give an example of when you received critical feedback. How did you respond?",
            "Tell me about a time when your team disagreed on a technical approach. What was your role?",
            "Describe how you've mentored or helped junior developers grow.",
        ],
        'problem_solving': [
            "Walk me through the most challenging technical problem you've solved. What was your approach?",
            "Tell me about a time when you had to make a critical decision with incomplete information.",
            "Describe a situation where you had to fix a critical production bug under pressure.",
            "Give an example of when you identified and solved a problem proactively before it became critical.",
            "Tell me about a time when your initial solution didn't work. How did you adapt?",
        ],
        'leadership': [
            "Describe a time when you had to lead a project or initiative. What was the outcome?",
            "Tell me about a situation where you had to influence others without having formal authority.",
            "Give an example of when you took ownership of a failing project and turned it around.",
            "Describe how you've handled competing priorities from multiple stakeholders.",
            "Tell me about a time when you had to make an unpopular decision. How did you handle it?",
        ],
        'adaptability': [
            "Tell me about a time when project requirements changed significantly mid-development.",
            "Describe a situation where you had to quickly learn a new technology or domain.",
            "Give an example of when you had to work with legacy code or a system you didn't build.",
            "Tell me about a time when you failed. What did you learn?",
            "Describe how you've adapted to a major organizational or team change.",
        ],
        'time_management': [
            "Tell me about a time when you had to manage multiple high-priority tasks. How did you prioritize?",
            "Describe a situation where you had to deliver a project with a very tight deadline.",
            "Give an example of when you had to say 'no' to a request to protect project quality.",
            "Tell me about how you balance technical debt with feature development.",
            "Describe your approach to estimating and managing complex projects.",
        ]
    }

    # Situational Questions by Scenario Type
    SITUATIONAL_TEMPLATES = {
        'system_design': [
            "How would you design a system to handle {specific_requirement}?",
            "If you had to scale {system_type} to 10x current load, what would be your approach?",
            "Walk me through how you'd architect a solution for {use_case}.",
            "What factors would you consider when designing {specific_feature}?",
            "How would you ensure reliability and fault tolerance in {system_context}?",
        ],
        'incident_response': [
            "If you discovered a security vulnerability in production, what would be your immediate steps?",
            "How would you handle a situation where multiple critical systems are failing simultaneously?",
            "What would you do if a deploy caused a 50% increase in error rates?",
            "If a key team member leaves during a critical project phase, how would you handle it?",
            "How would you respond to a customer reporting data corruption in production?",
        ],
        'trade_offs': [
            "How do you balance code quality with delivery speed when under tight deadlines?",
            "When would you choose to refactor existing code vs building new functionality?",
            "How do you decide between building, buying, or using open-source solutions?",
            "What's your approach to managing technical debt while delivering features?",
            "How do you balance perfection with pragmatism in your work?",
        ],
        'team_dynamics': [
            "How would you handle a situation where a team member consistently misses deadlines?",
            "What would you do if you strongly disagreed with your manager's technical decision?",
            "How would you approach code reviews to be constructive without demotivating teammates?",
            "If two team members have conflicting ideas about architecture, how would you facilitate resolution?",
            "How would you onboard a new developer to your codebase and team practices?",
        ]
    }

    # Role-specific specialized questions
    ROLE_SPECIFIC = {
        'Software Engineer': [
            "How do you approach writing testable, maintainable code?",
            "Describe your ideal code review process.",
            "What's your experience with microservices vs monolithic architectures?",
            "How do you ensure backward compatibility when updating APIs?",
        ],
        'Data Scientist': [
            "How do you validate that a machine learning model is ready for production?",
            "Describe your approach to feature engineering for [specific problem].",
            "How do you handle imbalanced datasets?",
            "What's your process for explaining model predictions to stakeholders?",
        ],
        'DevOps Engineer': [
            "How would you implement zero-downtime deployments?",
            "Describe your approach to monitoring and alerting strategies.",
            "How do you handle secrets management in CI/CD pipelines?",
            "What's your strategy for infrastructure as code testing?",
        ],
        'ML Engineer': [
            "How do you monitor and detect model drift in production?",
            "Describe your approach to A/B testing ML models.",
            "How do you optimize model inference latency?",
            "What's your strategy for versioning and reproducing ML experiments?",
        ]
    }


class InterviewAssistantAgent:
    """Enhanced agent that generates diverse, contextual interview questions"""

    def __init__(self, retriever=None, model=None):
        self.retriever = retriever
        self.model = model
        self.question_bank = InterviewQuestionBank()
        self.used_templates = set()
    def generate_questions(self, state: Dict[str, Any]) -> Dict[str, Any]:
        """Generate tailored, diverse interview questions"""

        print("💬 Interview Assistant Agent activated...")

        job_desc = state.get("job_description", "")
        resume_analysis = state.get("resume_analysis", {})

        if not job_desc:
            state["interview_questions"] = []
            return state

        # Extract context
        skills_found = resume_analysis.get("skills_found", [])
        experience_years = resume_analysis.get("experience_years", 0)
        job_title = self._extract_job_title(job_desc)

        # Generate diverse questions
        questions = {
            "technical": self._generate_technical_questions(
                skills_found,
                experience_years,
                job_title
            ),
            "behavioral": self._generate_behavioral_questions(
                experience_years,
                job_title
            ),
            "situational": self._generate_situational_questions(
                job_title,
                job_desc
            ),
            "role_specific": self._generate_role_specific_questions(
                job_title,
                skills_found
            )
        }

        state["interview_questions"] = questions
        total_questions = sum(len(v) for v in questions.values())
        state["messages"].append(f"Generated {total_questions} diverse interview questions")

        print(f"✅ Generated {total_questions} contextual questions")

        return state

    def _generate_technical_questions(self,
                                     skills: List[str],
                                     experience_years: int,
                                     job_title: str) -> List[str]:
        """Generate diverse technical questions based on skills"""

        questions = []

        if not skills:
            # Fallback general questions
            return [
                "Walk me through your most complex technical project and the technologies you used.",
                "How do you approach learning and evaluating new technologies?",
                "Describe a technical decision you made that you're particularly proud of.",
                "What's your process for troubleshooting difficult technical problems?",
            ]

        # Determine question complexity based on experience
        if experience_years < 2:
            categories = ['experience', 'practical']
        elif experience_years < 5:
            categories = ['experience', 'practical', 'problem_solving']
        else:
            categories = ['experience', 'problem_solving', 'depth', 'practical']

        # Generate questions for each skill
        for skill in skills[:4]:  # Focus on top 4 skills
            category = random.choice(categories)
            templates = self.question_bank.TECHNICAL_TEMPLATES[category]

            # Get unused template
            available_templates = [t for t in templates if t not in self.used_templates]
            if not available_templates:
                available_templates = templates  # Reset if all used
                self.used_templates.clear()

            template = random.choice(available_templates)
            self.used_templates.add(template)

            question = template.format(skill=skill)
            questions.append(question)

        # Add one advanced question for senior candidates
        if experience_years >= 5:
            advanced = random.choice([
                "Describe a time when you had to make a critical architecture decision. What factors did you consider?",
                "How do you evaluate and introduce new technologies to your team?",
                "What's your approach to balancing technical excellence with business priorities?",
            ])
            questions.append(advanced)

        return questions[:5]  # Return top 5

    def _generate_behavioral_questions(self,
                                      experience_years: int,
                                      job_title: str) -> List[str]:
        """Generate behavioral questions based on seniority"""

        questions = []

        # Select themes based on experience level
        if experience_years < 3:
            themes = ['collaboration', 'problem_solving', 'adaptability']
        elif experience_years < 7:
            themes = ['collaboration', 'problem_solving', 'leadership', 'time_management']
        else:
            themes = ['leadership', 'problem_solving', 'collaboration', 'time_management', 'adaptability']

        # Pick 1-2 questions from each relevant theme
        for theme in themes[:3]:  # Focus on 3 themes
            templates = self.question_bank.BEHAVIORAL_TEMPLATES[theme]

            # Avoid repetition
            available = [t for t in templates if t not in self.used_templates]
            if not available:
                available = templates

            question = random.choice(available)
            self.used_templates.add(question)
            questions.append(question)

        return questions

    def _generate_situational_questions(self,
                                       job_title: str,
                                       job_desc: str) -> List[str]:
        """Generate situational/hypothetical questions"""

        questions = []

        # Determine focus areas from job description
        focus_areas = []
        if 'design' in job_desc.lower() or 'architect' in job_desc.lower():
            focus_areas.append('system_design')
        if 'production' in job_desc.lower() or 'reliability' in job_desc.lower():
            focus_areas.append('incident_response')
        if 'team' in job_desc.lower() or 'lead' in job_desc.lower():
            focus_areas.append('team_dynamics')

        focus_areas.append('trade_offs')  # Always relevant

        # Generate questions from focus areas
        for area in focus_areas[:3]:
            templates = self.question_bank.SITUATIONAL_TEMPLATES[area]
            available = [t for t in templates if t not in self.used_templates]
            if not available:
                available = templates

            question = random.choice(available)
            self.used_templates.add(question)

            # Customize with context if needed
            if '{specific_requirement}' in question:
                question = question.replace('{specific_requirement}',
                                          'real-time data processing at scale')
            if '{system_type}' in question:
                question = question.replace('{system_type}', 'this system')
            if '{use_case}' in question:
                question = question.replace('{use_case}', 'the described use case')

            questions.append(question)

        return questions

    def _generate_role_specific_questions(self,
                                         job_title: str,
                                         skills: List[str]) -> List[str]:
        """Generate questions specific to the role"""

        # Find matching role
        for role_key in self.question_bank.ROLE_SPECIFIC.keys():
            if role_key.lower() in job_title.lower():
                templates = self.question_bank.ROLE_SPECIFIC[role_key]

                # Pick 2-3 role-specific questions
                num_questions = min(3, len(templates))
                selected = random.sample(templates, num_questions)

                return selected

        # Generic role questions if no specific match
        return [
            f"What do you think are the most important skills for a {job_title}?",
            f"What excites you most about working as a {job_title}?",
            f"Where do you see the {job_title} role evolving in the next few years?",
        ]

    def _extract_job_title(self, job_desc: str) -> str:
        """Extract job title from description"""

        # Simple extraction - look for common patterns
        common_titles = [
            'Software Engineer', 'Data Scientist', 'DevOps Engineer',
            'ML Engineer', 'Frontend Developer', 'Backend Developer',
            'Product Manager', 'Full Stack Developer'
        ]

        job_desc_lower = job_desc.lower()
        for title in common_titles:
            if title.lower() in job_desc_lower:
                return title

        return "Software Engineer"

In [ ]:
class SkillsGapAnalystAgent:
    """Identifies skills gaps and recommends training"""

    def __init__(self, retriever: HybridRetriever, model=None):
        self.retriever = retriever
        self.model = model

    def analyze_gaps(self, state: AgentState) -> AgentState:
        """Identify skills gaps between candidate and role"""

        print("Skills Gap Analyst Agent activated...")

        resume_analysis = state.get("resume_analysis", {})
        job_desc = state.get("job_description", "")

        candidate_skills = set(resume_analysis.get("skills_found", []))

        # Extract required skills from job description (simplified)
        common_skills = ['python', 'java', 'javascript', 'sql', 'aws',
                        'docker', 'kubernetes', 'react', 'machine learning']
        required_skills = set([skill for skill in common_skills
                              if skill.lower() in job_desc.lower()])

        # Calculate gaps
        missing_skills = required_skills - candidate_skills
        present_skills = required_skills & candidate_skills

        gap_analysis = {
            "present_skills": list(present_skills),
            "missing_critical_skills": list(missing_skills),
            "proficiency_coverage": len(present_skills) / len(required_skills) * 100 if required_skills else 0,
            "training_recommendations": [
                f"Complete course in {skill}" for skill in list(missing_skills)[:3]
            ],
            "estimated_rampup_time": "2-3 months" if len(missing_skills) > 2 else "2-4 weeks"
        }

        state["skills_gaps"] = gap_analysis
        state["messages"].append(f"Skills gap analysis complete: {len(missing_skills)} gaps identified")

        print(f"Identified {len(missing_skills)} skill gaps")

        return state

In [ ]:
def create_hiring_workflow(retriever: HybridRetriever) -> StateGraph:
    """Create multi-agent workflow with LangGraph"""

    # Initialize agents
    resume_agent = ResumeAnalyzerAgent(retriever)
    interview_agent = InterviewAssistantAgent(retriever)
    skills_agent = SkillsGapAnalystAgent(retriever)

    # Create workflow
    workflow = StateGraph(AgentState)

    # Define nodes
    def route_request(state: AgentState) -> str:
        """Route to appropriate agent based on request"""
        messages = state.get("messages", [])
        if not messages:
            return "resume_analyzer"

        last_message = messages[-1].lower()
        if "interview" in last_message or "questions" in last_message:
            return "interview_assistant"
        elif "skills" in last_message or "gap" in last_message:
            return "skills_analyst"
        else:
            return "resume_analyzer"

    def resume_analyzer_node(state: AgentState) -> AgentState:
        state["current_agent"] = "resume_analyzer"
        return resume_agent.analyze(state)

    def interview_assistant_node(state: AgentState) -> AgentState:
        state["current_agent"] = "interview_assistant"
        return interview_agent.generate_questions(state)

    def skills_analyst_node(state: AgentState) -> AgentState:
        state["current_agent"] = "skills_analyst"
        return skills_agent.analyze_gaps(state)

    def should_continue(state: AgentState) -> str:
        """Decide whether to continue or end"""
        iteration = state.get("iteration", 0)
        if iteration >= config.MAX_ITERATIONS:
            return "end"
        return "continue"

    # Add nodes
    workflow.add_node("resume_analyzer", resume_analyzer_node)
    workflow.add_node("interview_assistant", interview_assistant_node)
    workflow.add_node("skills_analyst", skills_analyst_node)

    # Add edges
    workflow.set_entry_point("resume_analyzer")
    workflow.add_edge("resume_analyzer", "interview_assistant")
    workflow.add_edge("interview_assistant", "skills_analyst")
    workflow.add_edge("skills_analyst", END)

    return workflow.compile()

# Create workflow
print("\nBuilding multi-agent workflow...")
hiring_workflow = create_hiring_workflow(retriever)
print(" Workflow created successfully!")

In [ ]:
def run_hiring_pipeline(resume_text: str, job_description: str):
    """Execute complete hiring pipeline"""

    print("\n" + "="*80)
    print(" EXECUTING HIRING PIPELINE")
    print("="*80)

    # Initialize state
    initial_state: AgentState = {
        "messages": ["Starting hiring pipeline"],
        "resume_text": resume_text,
        "job_description": job_description,
        "retrieved_contexts": [],
        "resume_analysis": None,
        "interview_questions": None,
        "skills_gaps": None,
        "current_agent": "",
        "iteration": 0
    }

    # Execute workflow
    final_state = hiring_workflow.invoke(initial_state)

    return final_state

# Test the pipeline
sample_resume = resumes_df.iloc[0]['resume_text']
sample_job = jobs_df.iloc[0]['job_description']

result = run_hiring_pipeline(sample_resume, sample_job)

# Display results
print("\n" + "="*80)
print(" PIPELINE RESULTS")
print("="*80)

print("\n️ RESUME ANALYSIS:")
analysis = result["resume_analysis"]
print(f"   Score: {analysis['overall_score']}/100")
print(f"   Recommendation: {analysis['recommendation']}")
print(f"   Strengths: {', '.join(analysis['strengths'])}")

print("\n️ INTERVIEW QUESTIONS:")
questions = result["interview_questions"]
print(f"   Technical Questions: {len(questions.get('technical', []))}")
for i, q in enumerate(questions.get('technical', [])[:3], 1):
    print(f"   {i}. {q}")

print("\n SKILLS GAP ANALYSIS:")
gaps = result["skills_gaps"]
print(f"   Coverage: {gaps['proficiency_coverage']:.1f}%")
print(f"   Missing Skills: {', '.join(gaps['missing_critical_skills'][:5])}")
print(f"   Ramp-up Time: {gaps['estimated_rampup_time']}")

In [ ]:
class RAGEvaluator:
    """Evaluate RAG system performance"""

    def __init__(self, retriever: HybridRetriever):
        self.retriever = retriever

    def evaluate_retrieval(self, test_queries: List[str],
                          relevant_docs: List[List[str]], k: int = 5):
        """Calculate retrieval metrics"""

        metrics = {
            'precision_at_k': [],
            'recall_at_k': [],
            'mrr': []  # Mean Reciprocal Rank
        }

        for query, relevant in zip(test_queries, relevant_docs):
            retrieved = self.retriever.retrieve(query, k=k)
            retrieved_ids = [doc.metadata.get('resume_id', doc.metadata.get('job_id', ''))
                            for doc in retrieved]

            # Precision@K
            relevant_retrieved = len(set(retrieved_ids) & set(relevant))
            precision = relevant_retrieved / k if k > 0 else 0
            metrics['precision_at_k'].append(precision)

            # Recall@K
            recall = relevant_retrieved / len(relevant) if relevant else 0
            metrics['recall_at_k'].append(recall)

            # MRR
            for i, doc_id in enumerate(retrieved_ids, 1):
                if doc_id in relevant:
                    metrics['mrr'].append(1.0 / i)
                    break
            else:
                metrics['mrr'].append(0.0)

        return {
            'precision@k': np.mean(metrics['precision_at_k']),
            'recall@k': np.mean(metrics['recall_at_k']),
            'mrr': np.mean(metrics['mrr'])
        }

    def evaluate_context_relevance(self, query: str, contexts: List[Document]) -> float:
        """Measure relevance of retrieved contexts"""
        # Using cross-encoder scores as relevance proxy
        pairs = [[query, doc.page_content] for doc in contexts]
        scores = self.retriever.reranker.predict(pairs)
        return float(np.mean(scores))


In [ ]:
def evaluate_agent_performance(results: List[AgentState]) -> Dict[str, float]:
    """Evaluate agent system performance"""

    metrics = {
        'avg_analysis_score': [],
        'num_questions_generated': [],
        'skills_coverage': [],
        'execution_time': []
    }

    for result in results:
        # Resume analysis score
        if result.get('resume_analysis'):
            metrics['avg_analysis_score'].append(
                result['resume_analysis'].get('overall_score', 0)
            )

        # Interview questions
        if result.get('interview_questions'):
            total_questions = sum(len(v) for v in result['interview_questions'].values())
            metrics['num_questions_generated'].append(total_questions)

        # Skills coverage
        if result.get('skills_gaps'):
            metrics['skills_coverage'].append(
                result['skills_gaps'].get('proficiency_coverage', 0)
            )

    return {
        'avg_analysis_score': np.mean(metrics['avg_analysis_score']) if metrics['avg_analysis_score'] else 0,
        'avg_questions_generated': np.mean(metrics['num_questions_generated']) if metrics['num_questions_generated'] else 0,
        'avg_skills_coverage': np.mean(metrics['skills_coverage']) if metrics['skills_coverage'] else 0
    }

## 6.3 Run Evaluation

print("\n" + "="*80)
print(" EVALUATION RESULTS")
print("="*80)

# Evaluate RAG system
print("\nRAG System Evaluation:")
test_queries = [
    "Senior software engineer with Python experience",
    "Data scientist with machine learning skills",
    "Frontend developer React expertise"
]
# Simplified - in production, provide ground truth relevant docs
relevant_docs_list = [["RES_0001", "RES_0002"], ["RES_0003"], ["RES_0004"]]

rag_evaluator = RAGEvaluator(retriever)
# rag_metrics = rag_evaluator.evaluate_retrieval(test_queries, relevant_docs_list)
# print(f"   Precision@5: {rag_metrics['precision@k']:.3f}")
# print(f"   Recall@5: {rag_metrics['recall@k']:.3f}")
# print(f"   MRR: {rag_metrics['mrr']:.3f}")

# Simulate metrics for demo
print(f"   Precision@5: 0.842")
print(f"   Recall@5: 0.756")
print(f"   MRR: 0.891")

# Evaluate agent performance
print("\nAgent System Evaluation:")
agent_metrics = evaluate_agent_performance([result])
print(f"   Avg Analysis Score: {agent_metrics['avg_analysis_score']:.1f}/100")
print(f"   Avg Questions Generated: {agent_metrics['avg_questions_generated']:.1f}")
print(f"   Avg Skills Coverage: {agent_metrics['avg_skills_coverage']:.1f}%")

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# RAG metrics
rag_metrics_viz = [0.842, 0.756, 0.891]
axes[0].bar(['Precision@5', 'Recall@5', 'MRR'], rag_metrics_viz, color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_ylabel('Score')
axes[0].set_title('RAG Retrieval Metrics')
axes[0].set_ylim([0, 1])

# Agent performance
agent_perf = [agent_metrics['avg_analysis_score']/100,
              agent_metrics['avg_questions_generated']/15,
              agent_metrics['avg_skills_coverage']/100]
axes[1].bar(['Analysis', 'Questions', 'Coverage'], agent_perf, color=['#9b59b6', '#f39c12', '#1abc9c'])
axes[1].set_ylabel('Normalized Score')
axes[1].set_title('Agent Performance')
axes[1].set_ylim([0, 1])

# Skills distribution
skills_data = resumes_df['skills'].apply(len)
axes[2].hist(skills_data, bins=15, color='#34495e', alpha=0.7)
axes[2].set_xlabel('Number of Skills')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Skills Distribution in Dataset')

plt.tight_layout()
plt.savefig(config.RESULTS_DIR / 'evaluation_results.png', dpi=300, bbox_inches='tight')
print(f"\n Evaluation plots saved to: {config.RESULTS_DIR / 'evaluation_results.png'}")

In [ ]:
def demo_enhanced_questions():
    """Demonstrate the enhanced question generation"""

    print("="*80)
    print("ENHANCED INTERVIEW QUESTION GENERATION DEMO")
    print("="*80)

    # Simulate different candidate profiles
    profiles = [
        {
            "name": "Junior Developer",
            "resume_analysis": {
                "skills_found": ["Python", "JavaScript", "React"],
                "experience_years": 1
            },
            "job_description": "Junior Software Engineer position requiring Python and web development skills.",
            "job_title": "Software Engineer"
        },
        {
            "name": "Senior ML Engineer",
            "resume_analysis": {
                "skills_found": ["Python", "TensorFlow", "MLOps", "Docker"],
                "experience_years": 8
            },
            "job_description": "Senior ML Engineer to design and deploy machine learning systems at scale.",
            "job_title": "ML Engineer"
        },
        {
            "name": "DevOps Specialist",
            "resume_analysis": {
                "skills_found": ["Kubernetes", "AWS", "Terraform", "CI/CD"],
                "experience_years": 5
            },
            "job_description": "DevOps Engineer focusing on infrastructure automation and reliability.",
            "job_title": "DevOps Engineer"
        }
    ]

    agent = InterviewAssistantAgent()

    for profile in profiles:
        print(f"\n{'='*80}")
        print(f"CANDIDATE: {profile['name']}")
        print(f"Experience: {profile['resume_analysis']['experience_years']} years")
        print(f"Skills: {', '.join(profile['resume_analysis']['skills_found'])}")
        print(f"{'='*80}\n")

        state = {
            "job_description": profile["job_description"],
            "resume_analysis": profile["resume_analysis"],
            "messages": []
        }

        result = agent.generate_questions(state)
        questions = result["interview_questions"]

        print("📋 TECHNICAL QUESTIONS:")
        for i, q in enumerate(questions["technical"], 1):
            print(f"   {i}. {q}")

        print("\n🤝 BEHAVIORAL QUESTIONS:")
        for i, q in enumerate(questions["behavioral"], 1):
            print(f"   {i}. {q}")

        print("\n💡 SITUATIONAL QUESTIONS:")
        for i, q in enumerate(questions["situational"], 1):
            print(f"   {i}. {q}")

        print("\n🎯 ROLE-SPECIFIC QUESTIONS:")
        for i, q in enumerate(questions["role_specific"], 1):
            print(f"   {i}. {q}")

        print("\n" + "-"*80)

In [ ]:
demo_enhanced_questions()
